In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime, timedelta
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.__version__

'1.5.2'

In [3]:
spark = SparkSession.builder.appName('CNEO_data_extractor').getOrCreate()
spark

In [17]:
date = datetime.today().date()
date_min = date + timedelta(days=59)
date_max = date + timedelta(days=60)
print(date_min, date_max)

2023-03-03 2023-03-04


In [5]:
# url = "https://ssd-api.jpl.nasa.gov/cad.api"
# params = {
#     "date-min": str(date),
#     "dist-max": "0.1",
#     'fullname': "true",
#     'dist-max': "0.1",
#     'diameter': "true"
# }
# response2 = requests.get(url, params)
# data2 = response2.json()


In [6]:
# data2['count']

In [3]:
url = "https://ssd-api.jpl.nasa.gov/cad.api"
parameters = {
    "date-min": "1900-01-04",
    "date-max": str(datetime.today().date()),
    "dist-max": "0.05",
    'fullname': "true",
    'dist-max': "0.1",
    'diameter': "true"
}
response = requests.get(url, parameters)
data = response.json()


In [9]:
columns = [
    'Designation',
    'Orbit Id',
    'Time of Close approach',
    'Close-Approach Date',
    'Nominal Approch distance (au)',
    'Min Close-Approach Distance (au)',
    'Max Close-Approach Distance (au)',
    'V Reletive (Km/s)',
    'V Infinite (Km/s)',
    'Close-Approach Uncertain Time',
    'Absolute Magnitude (mag)',
    'Diameter (Km)',
    'Diameter-Sigma (Km)',
    'Object'
]
# types = [
#     StringType(),
#     StringType(),
#     DoubleType(),
#     DateType(),
#     DoubleType(),
#     DoubleType(),
#     DoubleType(),
#     DoubleType(),
#     DoubleType(),
#     StringType(),
#     DoubleType(),
#     DoubleType(),
#     DoubleType(),
#     StringType(),
#          ]
# dic = {columns[i]: types[i] for i in range(len(columns))}
# dic

In [26]:
df = pd.DataFrame(np.array(data['data']), columns = data['fields'])
df.head()

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h,diameter,diameter_sigma,fullname
0,2020 BN7,6,2415023.594589649,1900-Jan-04 02:16,0.0896607474147164,0.0882582365913522,0.0914306781836958,5.2581158476114,5.25246109718832,1_13:09,23.8,None,None,(2020 BN7)
1,2017 MW4,18,2415023.595882106,1900-Jan-04 02:18,0.0613004997707699,0.0612907488305461,0.0613102521172838,17.5916418448861,17.5891708464552,00:01,20.05,None,None,(2017 MW4)
2,509352,57,2415024.433789572,1900-Jan-04 22:25,0.00963183861698794,0.00962494789186613,0.00963873093215005,8.68671104177175,8.65480697513416,00:02,20.16,None,None,509352 (2007 AG)
3,214869,173,2415026.944065756,1900-Jan-07 10:39,0.0501688442324355,0.0501602845857579,0.0501774043027291,11.5943349372234,11.5897533259902,00:02,16.51,None,None,214869 (2007 PA8)
4,2017 QD3,7,2415029.500845714,1900-Jan-10 00:01,0.0865599287526722,0.08654753449196,0.0865723230832191,10.9317273117942,10.928911117456,00:03,22.23,None,None,(2017 QD3)


In [27]:
df['jd'] = pd.to_numeric(df['jd'])
df['cd'] = pd.to_datetime(df['cd'])
df['dist'] = pd.to_numeric(df['dist'])
df['dist_min'] = pd.to_numeric(df['dist_min'])
df['dist_max'] = pd.to_numeric(df['dist_max'])
df['v_rel'] = pd.to_numeric(df['v_rel'])
df['v_inf'] = pd.to_numeric(df['v_inf'])
df['t_sigma_f'] = df['t_sigma_f'].astype(str)
df['h'] = pd.to_numeric(df['h'])
df['diameter'] = pd.to_numeric(df['diameter'])
df['diameter_sigma'] = pd.to_numeric(df['diameter_sigma'])
df.tail()

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h,diameter,diameter_sigma,fullname
46777,2022 YU3,5,2.459947e+06,2023-01-02 05:11:00,0.025191,0.025017,0.025365,7.133786,7.118944,< 00:01,25.835,NaN,NaN,(2022 YU3)
46778,2022 YZ3,3,2.459947e+06,2023-01-02 09:29:00,0.060599,0.060118,0.061080,11.704659,11.700902,< 00:01,25.088,NaN,NaN,(2022 YZ3)
46779,2022 YY6,4,2.459947e+06,2023-01-02 11:07:00,0.005439,0.005411,0.005467,20.269551,20.245368,< 00:01,26.109,NaN,NaN,(2022 YY6)
46780,2022 YP5,6,2.459947e+06,2023-01-02 17:53:00,0.019977,0.019919,0.020036,4.793711,4.765807,< 00:01,27.085,NaN,NaN,(2022 YP5)
46781,2021 NF,6,2.459947e+06,2023-01-02 21:51:00,0.045947,0.039555,0.052455,11.326777,11.321656,03:09,24.730,NaN,NaN,(2021 NF)


In [7]:
df = pd.read_csv('./raw_input_data.csv')
df

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h,diameter,diameter_sigma,fullname
0,2020 BN7,6,2.415024e+06,1900-01-04 02:16:00,0.089661,0.088258,0.091431,5.258116,5.252461,1_13:09,23.800,NaN,NaN,(2020 BN7)
1,2017 MW4,18,2.415024e+06,1900-01-04 02:18:00,0.061300,0.061291,0.061310,17.591642,17.589171,00:01,20.050,NaN,NaN,(2017 MW4)
2,509352,57,2.415024e+06,1900-01-04 22:25:00,0.009632,0.009625,0.009639,8.686711,8.654807,00:02,20.160,NaN,NaN,509352 (2007 AG)
3,214869,173,2.415027e+06,1900-01-07 10:39:00,0.050169,0.050160,0.050177,11.594335,11.589753,00:02,16.510,NaN,NaN,214869 (2007 PA8)
4,2017 QD3,7,2.415030e+06,1900-01-10 00:01:00,0.086560,0.086548,0.086572,10.931727,10.928911,00:03,22.230,NaN,NaN,(2017 QD3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46777,2022 YU3,5,2.459947e+06,2023-01-02 05:11:00,0.025191,0.025017,0.025365,7.133786,7.118944,< 00:01,25.835,NaN,NaN,(2022 YU3)
46778,2022 YZ3,3,2.459947e+06,2023-01-02 09:29:00,0.060599,0.060118,0.061080,11.704659,11.700902,< 00:01,25.088,NaN,NaN,(2022 YZ3)
46779,2022 YY6,4,2.459947e+06,2023-01-02 11:07:00,0.005439,0.005411,0.005467,20.269551,20.245368,< 00:01,26.109,NaN,NaN,(2022 YY6)
46780,2022 YP5,6,2.459947e+06,2023-01-02 17:53:00,0.019977,0.019919,0.020036,4.793711,4.765807,< 00:01,27.085,NaN,NaN,(2022 YP5)


In [10]:
df.columns = columns
df

,Designation,Orbit Id,Time of Close approach,Close-Approach Date,Nominal Approch distance (au),Min Close-Approach Distance (au),Max Close-Approach Distance (au),V Reletive (Km/s),V Infinite (Km/s),Close-Approach Uncertain Time,Absolute Magnitude (mag),Diameter (Km),Diameter-Sigma (Km),Object
0,2020 BN7,6,2.415024e+06,1900-01-04 02:16:00,0.089661,0.088258,0.091431,5.258116,5.252461,1_13:09,23.800,NaN,NaN,(2020 BN7)
1,2017 MW4,18,2.415024e+06,1900-01-04 02:18:00,0.061300,0.061291,0.061310,17.591642,17.589171,00:01,20.050,NaN,NaN,(2017 MW4)
2,509352,57,2.415024e+06,1900-01-04 22:25:00,0.009632,0.009625,0.009639,8.686711,8.654807,00:02,20.160,NaN,NaN,509352 (2007 AG)
3,214869,173,2.415027e+06,1900-01-07 10:39:00,0.050169,0.050160,0.050177,11.594335,11.589753,00:02,16.510,NaN,NaN,214869 (2007 PA8)
4,2017 QD3,7,2.415030e+06,1900-01-10 00:01:00,0.086560,0.086548,0.086572,10.931727,10.928911,00:03,22.230,NaN,NaN,(2017 QD3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46777,2022 YU3,5,2.459947e+06,2023-01-02 05:11:00,0.025191,0.025017,0.025365,7.133786,7.118944,< 00:01,25.835,NaN,NaN,(2022 YU3)
46778,2022 YZ3,3,2.459947e+06,2023-01-02 09:29:00,0.060599,0.060118,0.061080,11.704659,11.700902,< 00:01,25.088,NaN,NaN,(2022 YZ3)
46779,2022 YY6,4,2.459947e+06,2023-01-02 11:07:00,0.005439,0.005411,0.005467,20.269551,20.245368,< 00:01,26.109,NaN,NaN,(2022 YY6)
46780,2022 YP5,6,2.459947e+06,2023-01-02 17:53:00,0.019977,0.019919,0.020036,4.793711,4.765807,< 00:01,27.085,NaN,NaN,(2022 YP5)


In [11]:
df.to_csv('./raw_input_data.csv', index=False)

In [28]:
# psdf = ps.DataFrame(np.array(data['data']))
# psdf.head()

In [21]:
sdf = spark.read.csv('./raw_input_data.csv', inferSchema=True, header=True)
sdf

DataFrame[Designation: string, Orbit Id: string, Time of Close approach: double, Close-Approach Date: timestamp, Nominal Approch distance (au): double, Min Close-Approach Distance (au): double, Max Close-Approach Distance (au): double, V Reletive (Km/s): double, V Infinite (Km/s): double, Close-Approach Uncertain Time: string, Absolute Magnitude (mag): double, Diameter (Km): double, Diameter-Sigma (Km): double, Object: string]

In [22]:
sdf.show(2)

+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|Designation|Orbit Id|Time of Close approach|Close-Approach Date|Nominal Approch distance (au)|Min Close-Approach Distance (au)|Max Close-Approach Distance (au)|V Reletive (Km/s)|V Infinite (Km/s)|Close-Approach Uncertain Time|Absolute Magnitude (mag)|Diameter (Km)|Diameter-Sigma (Km)|           Object|
+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|   2020 BN7|       6|     2415023.594589649|1900-01-04 02:16:00|           0.0896607

In [23]:
sdf = sdf.fillna(value=-1)

In [24]:
sdf.show(2)

+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|Designation|Orbit Id|Time of Close approach|Close-Approach Date|Nominal Approch distance (au)|Min Close-Approach Distance (au)|Max Close-Approach Distance (au)|V Reletive (Km/s)|V Infinite (Km/s)|Close-Approach Uncertain Time|Absolute Magnitude (mag)|Diameter (Km)|Diameter-Sigma (Km)|           Object|
+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|   2020 BN7|       6|     2415023.594589649|1900-01-04 02:16:00|           0.0896607

In [72]:
# psdf = ps.DataFrame(
#     {'a': [1, 2, 3, 4, 5, 6],
#      'b': [100, 200, 300, 400, 500, 600],
#      'c': ["one", "two", "three", "four", "five", "six"]})

In [11]:
df.columns = columns
df

,Designation,Orbit Id,Time of Close approach,Close-Approach Date,Nominal Approch distance (au),Min Close-Approach Distance (au),Max Close-Approach Distance (au),V Reletive (Km/s),V Infinite (Km/s),Close-Approach Uncertain Time,Absolute Magnitude (mag),Diameter (Km),Diameter-Sigma (Km),Object
0,2020 BN7,6,2.415024e+06,1900-01-04 02:16:00,0.089661,0.088258,0.091431,5.258116,5.252461,1_13:09,23.800,NaN,NaN,(2020 BN7)
1,2017 MW4,18,2.415024e+06,1900-01-04 02:18:00,0.061300,0.061291,0.061310,17.591642,17.589171,00:01,20.050,NaN,NaN,(2017 MW4)
2,509352,57,2.415024e+06,1900-01-04 22:25:00,0.009632,0.009625,0.009639,8.686711,8.654807,00:02,20.160,NaN,NaN,509352 (2007 AG)
3,214869,173,2.415027e+06,1900-01-07 10:39:00,0.050169,0.050160,0.050177,11.594335,11.589753,00:02,16.510,NaN,NaN,214869 (2007 PA8)
4,2017 QD3,7,2.415030e+06,1900-01-10 00:01:00,0.086560,0.086548,0.086572,10.931727,10.928911,00:03,22.230,NaN,NaN,(2017 QD3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46777,2022 YU3,5,2.459947e+06,2023-01-02 05:11:00,0.025191,0.025017,0.025365,7.133786,7.118944,< 00:01,25.835,NaN,NaN,(2022 YU3)
46778,2022 YZ3,3,2.459947e+06,2023-01-02 09:29:00,0.060599,0.060118,0.061080,11.704659,11.700902,< 00:01,25.088,NaN,NaN,(2022 YZ3)
46779,2022 YY6,4,2.459947e+06,2023-01-02 11:07:00,0.005439,0.005411,0.005467,20.269551,20.245368,< 00:01,26.109,NaN,NaN,(2022 YY6)
46780,2022 YP5,6,2.459947e+06,2023-01-02 17:53:00,0.019977,0.019919,0.020036,4.793711,4.765807,< 00:01,27.085,NaN,NaN,(2022 YP5)


In [49]:
# def createSchema(columns):
#     myScehma = []
#     for key, value in dic.items():
#         t = StructField(key, value, True)
#         myScehma.append(t)
#     # print(myScehma)
#     return myScehma
# scheme = createSchema(columns)
# # print(type(scheme))
# myScehma = StructType([i for i in scheme])
# myScehma


In [50]:
# with open('data.json', 'w') as datafile:
#     json.dump(data['data'], datafile)

In [24]:
# psdf = ps.from_pandas(df)

In [12]:
# sdf = spark.read.schema(myScehma).json('./data.json')
# sdf.printSchema()

sdf = spark.createDataFrame(df)


c:\Users\vaibh\anaconda3\envs\dataengineer\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\vaibh\anaconda3\envs\dataengineer\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


OverflowError: mktime argument out of range

In [9]:
sdf.show()

Py4JJavaError: An error occurred while calling o46.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.sc

In [62]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
sdf = spark.createDataFrame(df)


c:\Users\vaibh\anaconda3\envs\dataengineer\lib\site-packages\pyspark\sql\pandas\conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
c:\Users\vaibh\anaconda3\envs\dataengineer\lib\site-packages\pyspark\sql\pandas\conversion.py:425: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert '6' with type str: tried to convert to int32
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


TypeError: field Orbit Id: IntegerType() can not accept object '6' in type <class 'str'>

In [56]:
sdf.printSchema()


root
 |-- Designation: string (nullable = true)
 |-- Orbit Id: string (nullable = true)
 |-- Time of Close approach: string (nullable = true)
 |-- Close-Approach Date: string (nullable = true)
 |-- Nominal Approch distance (au): string (nullable = true)
 |-- Min Close-Approach Distance (au): string (nullable = true)
 |-- Max Close-Approach Distance (au): string (nullable = true)
 |-- V Reletive (Km/s): string (nullable = true)
 |-- V Infinite (Km/s): string (nullable = true)
 |-- Close-Approach Uncertain Time: string (nullable = true)
 |-- Absolute Magnitude (mag): string (nullable = true)
 |-- Diameter (Km): string (nullable = true)
 |-- Diameter-Sigma (Km): string (nullable = true)
 |-- Object: string (nullable = true)



In [ ]:

import pandas as pd    
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54],['Ann',34]] 
  
# Create the pandas DataFrame 
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
  
# print dataframe. 
print(pandasDF)

# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .master("local[1]") \
#     .appName("SparkByExamples.com") \
#     .getOrCreate()

sparkDF=spark.createDataFrame(pandasDF) 
sparkDF.printSchema()
sparkDF.show()

#sparkDF=spark.createDataFrame(pandasDF.astype(str)) 
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
mySchema = StructType([ StructField("First Name", StringType(), True)\
                       ,StructField("Age", IntegerType(), True)])

sparkDF2 = spark.createDataFrame(pandasDF,schema=mySchema)
sparkDF2.printSchema()
sparkDF2.show()

# Enable Apache Arrow to convert Pandas to PySpark DataFrame
spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkDF2=spark.createDataFrame(pandasDF) 
sparkDF2.printSchema()
sparkDF2.show()

#Convert PySpark DataFrame to Pandas
pandasDF2=sparkDF2.select("*").toPandas
print(pandasDF2)



In [36]:
sdf.show(5)

Py4JJavaError: An error occurred while calling o146.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.s

In [87]:
lunar_distance_multiplier = 389.174

In [1]:
num1 = 50
num2 = 2
num3 = 3
num4 = 8
result = num1/num4-num3*num2+num4
print(result)

8.25


In [3]:
for number in range(1,5):
    print ("The current number is ",number)
print("---------------------------")
for number in range(1,7,2):
    print ("The current number is ",number)
print("---------------------------")
for number in range(5,0,-1):
    print ("The current number is ",number)


The current number is  1
The current number is  2
The current number is  3
The current number is  4
---------------------------
The current number is  1
The current number is  3
The current number is  5
---------------------------
The current number is  5
The current number is  4
The current number is  3
The current number is  2
The current number is  1


In [4]:
for number in 10, 15:
    for counter in range(1, 3):
        print(number*counter, end=" ")


10 20 15 30 

In [5]:
for num in 23, 45, 50, 65, 76, 90:
    if(num % 5 != 0):
        continue
    if(num % 10 == 0):
        print(num, end=" ")
        continue
    if(num % 3 == 0):
        print(num, end=" ")


45 50 90 

In [6]:
number=28
for num in range(25,30):
    if(number>num):
        print(num)
    else:
        print(num)
        break

25
26
27
28


In [7]:
def change_number(num):
    num+=10
def change_list(num_list):
    num_list.append(20)
num_val=10
print("*********effect of pass by value*********")
print("num_val before function call:", num_val)
change_number(num_val)
print("num_val after function call:", num_val)
print("-----------------------------------------------")
val_list=[5,10,15]
print("*********effect of pass by reference*********")
print("val_list before function call:", val_list)
change_list(val_list)
print("val_list after function call:", val_list)


*********effect of pass by value*********
num_val before function call: 10
num_val after function call: 10
-----------------------------------------------
*********effect of pass by reference*********
val_list before function call: [5, 10, 15]
val_list after function call: [5, 10, 15, 20]


In [8]:
def display1(flight_number, seating_capacity):
    print("Flight Number:", flight_number)
    print("Seating Capacity:", seating_capacity)
print("code-1: positional arguments")
display1("FN789",200)
#Uncomment and execute the below function call statement and observe the output
#display1(300,"FN123")
def display2(flight_number, seating_capacity):
    print("Flight Number:", flight_number)
    print("Seating Capacity:", seating_capacity)
print("-------------------------------------------------")
print("code-2: keyword arguments")
display2(seating_capacity=250, flight_number="FN789")
def display3(flight_number, flight_make="Boeing", seating_capacity=150):
    print("Flight Number:", flight_number)
    print("Flight Make:", flight_make)
    print("Seating Capacity:", seating_capacity)
print("-------------------------------------------------")
print("code-3: default arguments")
display3("FN789","Eagle")
#Uncomment and execute the below function call statements one by one and observe the output
#display3("FN234")
#display3("FN678","Qantas",200)
def display4(passenger_name, *baggage_tuple):
    print("Passenger name:",passenger_name)
    total_wt=0
    for baggage_wt in baggage_tuple:
        total_wt+=baggage_wt
    print("Total baggage weight in kg:", total_wt)
print("-------------------------------------------------")
print("code-4: variable argument count")
display4("Jack",12,8,5)
#Uncomment and execute the below function call statements one by one and observe the output
#display4("Chan",20,12)
#display4("Henry",23)


code-1: positional arguments
Flight Number: FN789
Seating Capacity: 200
-------------------------------------------------
code-2: keyword arguments
Flight Number: FN789
Seating Capacity: 250
-------------------------------------------------
code-3: default arguments
Flight Number: FN789
Flight Make: Eagle
Seating Capacity: 150
-------------------------------------------------
code-4: variable argument count
Passenger name: Jack
Total baggage weight in kg: 25


In [9]:
message="welcome to Mysore"
word=message[-7:]
if(word=="Mysore"):
    print("got it")
else:
    message=message[3:14]
    print(message)

come to Mys


In [12]:
import math
num1=234.01
num2=6
num3=-27.01
print("The smallest integer greater than or equal to num1,",num1,":",math.ceil(num1))
print("The largest integer smaller than or equal to num1,",num1,":",math.floor(num1))
print("The factorial of num2,",num2,":", math.factorial(num2))
print("The absolute value of num3",num3,":",math.fabs(num3))


The smallest integer greater than or equal to num1, 234.01 : 235
The largest integer smaller than or equal to num1, 234.01 : 234
The factorial of num2, 6 : 720
The absolute value of num3 -27.01 : 27.01


In [13]:
crew_details={
    "Pilot":"Kumar",
    "Co-pilot":"Raghav",
    "Head-Strewardess":"Malini",
    "Stewardess":"Mala"
}
print("Before update:")
print("Co-pilot:",crew_details.get("Co-pilot"))
crew_details.update({"Flight Attendant":"Jane", "Co-pilot":"Henry"})
print("\nAfter update:")
print("Co-pilot:",crew_details.get("Co-pilot"))
print("Flight Attendant:",crew_details["Flight Attendant"])


Before update:
Co-pilot: Raghav

After update:
Co-pilot: Henry
Flight Attendant: Jane


In [17]:
A = [1, 2, 3, 4, 5, 6]
A.pop(2)

3

In [18]:
A

[1, 2, 4, 5, 6]

In [21]:
import re
if(re.search(r"Air","Airline")!=None):
    print("Pattern found")
else:
    print("Pattern not found")


Pattern found


In [22]:
song="JINGLE Bells jingle Bells Jingle All The Way"
song.upper()
song_words=song.split()
count=0
for word in song_words:
    if(word.startswith("jingle")):
        count=count+1
print(count)

1


In [23]:
sample_dict={'a':1,'b':2}
sample_dict.update({'b':5, 'c':10 })
print(sample_dict.get('a'),sample_dict.get('b'),sample_dict.get('c'))

 


1 5 10


In [24]:
import re
word="New Airlines4"
if(re.search(r"^N",word) and re.search(r"e$",word)):
    print(re.sub(r"New",r"Old",word))
else:
    print(re.sub(r"s(\d{1})",r"S\1",word))

New AirlineS4


In [25]:
import math
num_list=[100.5,30.465,-1.22,20.15]
num_list.insert(1, -100.5)
num_list.pop(0)
num_list.sort()
print(math.ceil(math.fabs(num_list[0])))

 


101
